# Computing statistics that can assist in creating an investment portfolio

We are interested in creating a 1-year portfolio as these futures expire within the near future and can't be held for too long.

In [29]:
import numpy as np
import pandas as pd
import yfinance as yf

In [30]:
PATH = 'processed_data/full_dataset.csv'

In [31]:
risk_free_stock = yf.Ticker('^IRX')
risk_free_stock = risk_free_stock.history(period='5y')
risk_free_stock.reset_index(inplace=True)
risk_free_stock['Date'] = risk_free_stock['Date'].dt.date
risk_free_stock = risk_free_stock[['Date', 'Close']]

In [32]:
stock_information = pd.read_csv(PATH)
separate_dfs = {}
for column in stock_information.columns:
    if column not in ['Date', 'diff', 'Prime']:
        separate_dfs[column] = stock_information[['Date', column]].rename(columns={column: 'Close'})
        separate_dfs[column] = separate_dfs[column].iloc[-252:]
    

In [33]:
risk_free_stock.dtypes

Date      object
Close    float64
dtype: object

In [34]:
# calculate the daily returns for each stock
for key in separate_dfs:
    separate_dfs[key]['Daily Returns'] = separate_dfs[key]['Close'].pct_change()
    separate_dfs[key].dropna(inplace=True)
    separate_dfs[key]['Date'] = pd.to_datetime(separate_dfs[key]['Date'])

# calculate the daily returns for the risk free stock
risk_free_stock['Daily Returns'] = risk_free_stock['Close'].pct_change()
risk_free_stock.dropna(inplace=True)
risk_free_stock['Date'] = pd.to_datetime(risk_free_stock['Date'])

In [35]:
separate_dfs['Maize Jul'].dtypes

Date             datetime64[ns]
Close                   float64
Daily Returns           float64
dtype: object

## Sharpe Ratio

In [36]:
# subtract daily returns of risk free stock from daily returns of each stock, match the dates
sharpe_dfs = {}
for key in separate_dfs:
    sharpe_dfs[key] = pd.merge(separate_dfs[key], risk_free_stock, on='Date', how='left')
    sharpe_dfs[key]['Excess Returns'] = sharpe_dfs[key]['Daily Returns_x'] - sharpe_dfs[key]['Daily Returns_y']

In [37]:
for key in sharpe_dfs:
    print(f'Sharpe Ratio for {key}: {sharpe_dfs[key]["Excess Returns"].mean() / sharpe_dfs[key]["Excess Returns"].std() * np.sqrt(len(sharpe_dfs[key]))}')

Sharpe Ratio for Maize Mar: -1.349088299903418
Sharpe Ratio for Maize May: -1.384392300292323
Sharpe Ratio for Maize Jul: -1.4205865377067177
Sharpe Ratio for Maize Sep: -1.3248755469378672
Sharpe Ratio for Oats Mar: -0.9746272060767236
Sharpe Ratio for Oats May: -1.0280654460066754
Sharpe Ratio for Oats Jul: -0.9057002136514066
Sharpe Ratio for Oats Sep: -0.9231922954440658
Sharpe Ratio for Soybeans Mar: -1.5696991164446301
Sharpe Ratio for Soybeans May: -1.5027798456972452
Sharpe Ratio for Soybeans Jul: -1.4588263806585897
Sharpe Ratio for Soybeans Sep: -1.3478167163800001
Sharpe Ratio for Wheat Mar: -0.8546812691828882
Sharpe Ratio for Wheat May: -0.8916929018523267
Sharpe Ratio for Wheat Jul: -0.9786382504443876
Sharpe Ratio for Wheat Sep: -1.0087996358251485


Negative Sharpe Ratio shows that it is unwise to invest in any of these stocks.

## Sortino Ratio

Similar to Sharpe, with different normalization.

In [38]:
for key in sharpe_dfs:
    sortino_ratio = sharpe_dfs[key]['Excess Returns'].mean() / sharpe_dfs[key]['Excess Returns'][sharpe_dfs[key]['Excess Returns'] < 0].std() * np.sqrt(len(sharpe_dfs[key]))
    print(f'Sortino Ratio for {key}: {sortino_ratio}')


Sortino Ratio for Maize Mar: -2.3979569244289363
Sortino Ratio for Maize May: -2.3705700568236425
Sortino Ratio for Maize Jul: -2.3393139577126525
Sortino Ratio for Maize Sep: -2.215687062453782
Sortino Ratio for Oats Mar: -1.4931529406880437
Sortino Ratio for Oats May: -1.584773981079656
Sortino Ratio for Oats Jul: -1.3795123867029842
Sortino Ratio for Oats Sep: -1.4112165873647535
Sortino Ratio for Soybeans Mar: -2.1627456810112187
Sortino Ratio for Soybeans May: -1.9892199012523053
Sortino Ratio for Soybeans Jul: -1.898851423916218
Sortino Ratio for Soybeans Sep: -1.8208290925719466
Sortino Ratio for Wheat Mar: -1.5935713710136574
Sortino Ratio for Wheat May: -1.6499647919438283
Sortino Ratio for Wheat Jul: -1.818541609483092
Sortino Ratio for Wheat Sep: -1.8948227453162838


## Beta

Computing the beta of each stock

In [39]:
market = yf.Ticker('^GSPC')
market = market.history(period='5y')
market.reset_index(inplace=True)
market['Date'] = market['Date'].dt.date
market = market[['Date', 'Close']]
market['Daily Returns'] = market['Close'].pct_change()
market.dropna(inplace=True)

In [40]:
market

,Date,Close,Daily Returns
1,2019-11-05,3074.620117,-0.001186
2,2019-11-06,3076.780029,0.000702
3,2019-11-07,3085.179932,0.002730
4,2019-11-08,3093.080078,0.002561
5,2019-11-11,3087.010010,-0.001962
...,...,...,...
1253,2024-10-28,5823.520020,0.002651
1254,2024-10-29,5832.919922,0.001614
1255,2024-10-30,5813.669922,-0.003300
1256,2024-10-31,5705.450195,-0.018615


## Treynor Ratio / Beta

The beta statistic of a stock shows its behavior compared to the behavior of the market (such as S&P 500).

In [42]:
jensen = {}
jensen['beta'] = {}

market_returns = market['Daily Returns']
risk_free = sharpe_dfs['Maize Jul']['Daily Returns_y']  

for key in sharpe_dfs:
    # Prepare the data for excess returns
    returns = sharpe_dfs[key]['Daily Returns_x']
    jensen[key] = pd.concat([returns, risk_free, market_returns], axis=1)
    
    jensen[key].columns = ['Stock Returns', 'Risk-Free Returns', 'Market Returns']
    jensen[key].dropna(inplace=True)

    # Calculate beta using covariance
    covariance = np.cov(jensen[key]['Stock Returns'], jensen[key]['Market Returns'])[0][1]
    market_variance = np.var(jensen[key]['Market Returns'])
    beta = covariance / market_variance if market_variance != 0 else np.nan
    jensen['beta'][key] = beta

    print(f'Beta for {key}: {beta}')
    
    


Beta for Maize Mar: 0.013062156325575952
Beta for Maize May: 0.011652710836097452
Beta for Maize Jul: 0.01009545513937746
Beta for Maize Sep: 0.01110909696991195
Beta for Oats Mar: 0.09868165958262176
Beta for Oats May: 0.09924490268194855
Beta for Oats Jul: 0.10994220146022665
Beta for Oats Sep: 0.08053785982636645
Beta for Soybeans Mar: 0.0025354826455161994
Beta for Soybeans May: 0.004417310888908352
Beta for Soybeans Jul: 0.004367596010293591
Beta for Soybeans Sep: 0.0028206694071068864
Beta for Wheat Mar: 0.044326910387352265
Beta for Wheat May: 0.044120848669302416
Beta for Wheat Jul: 0.045976875978192853
Beta for Wheat Sep: 0.045617313837664916


The beta values are very low ($\approx 0$), making the Treynor Ratio not informative.

## Jensen Alpha

Jensen's alpha measures a portfolio's performance relative to its expected return under the Capital Asset Pricing Model (CAPM), given the portfolio’s risk (beta) and the market’s return. A positive alpha indicates that the portfolio outperformed its expected return, suggesting superior management or strategy, while a negative alpha suggests underperformance.

In [43]:
for key in jensen:
    if key != 'beta':
        alpha = jensen[key]['Stock Returns'].mean() - (jensen['beta'][key] * (jensen[key]['Market Returns'].mean() - jensen[key]['Risk-Free Returns'].mean()))
        print(f'Alpha for {key}: {alpha}')

Alpha for Maize Mar: -0.0010846890580419812
Alpha for Maize May: -0.00103235066869714
Alpha for Maize Jul: -0.0009847264275966831
Alpha for Maize Sep: -0.0008063597123817832
Alpha for Oats Mar: -0.0013395389410916958
Alpha for Oats May: -0.0013740270750800083
Alpha for Oats Jul: -0.0012412554348414252
Alpha for Oats Sep: -0.001236962259796278
Alpha for Soybeans Mar: -0.001074619219256402
Alpha for Soybeans May: -0.0010102847520905675
Alpha for Soybeans Jul: -0.0009615220444438624
Alpha for Soybeans Sep: -0.0008092690182803176
Alpha for Wheat Mar: -0.0009898469565548947
Alpha for Wheat May: -0.0009650565381352522
Alpha for Wheat Jul: -0.000953807645708811
Alpha for Wheat Sep: -0.0009145521762382163
